In [1]:
import os
import shutil
import sys
import win32com

# Pfad zum Cache finden
gen_py_path = os.path.join(win32com.__gen_path__)

print(f"Versuche COM-Cache zu löschen in: {gen_py_path}")

if os.path.exists(gen_py_path):
    try:
        shutil.rmtree(gen_py_path)
        print("✅ Cache erfolgreich gelöscht! Bitte Kernel neu starten (Restart Kernel).")
    except PermissionError:
        print("❌ Zugriff verweigert. Bitte schließe alle Python/Jupyter Prozesse und lösche den Ordner manuell.")
    except Exception as e:
        print(f"❌ Fehler: {e}")
else:
    print("ℹ️ Kein Cache gefunden (das ist gut).")

Versuche COM-Cache zu löschen in: C:\Users\ALEXAN~1\AppData\Local\Temp\gen_py\3.12
ℹ️ Kein Cache gefunden (das ist gut).


In [4]:
import os
import time
import uuid
import xml.etree.ElementTree as ET
import win32com.client


# =========================
# 1) KONFIGURATION
# =========================

SLN_PATH = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln"

# Deine Quelle (TcPlcObject XML)
SOURCE_TC_OBJECT_XML = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\FB_GeneratedJob_01.xml"

# Zielordner (NICHT OneDrive)
OUT_DIR = r"D:\MA_Python_Agent\Notebooks\Generierte_PLCOpenXML"

# Du kannst den Namen gern einfach .xml nennen
OUT_FILE_NAME = "FB_GeneratedJob_01.xml"
OUT_XML_PATH = os.path.join(OUT_DIR, OUT_FILE_NAME)

DTE_PROGID = "TcXaeShell.DTE.17.0"


# PLCIMPORTOPTIONS
PLCIMPORTOPTIONS_NONE = 0
PLCIMPORTOPTIONS_RENAME = 1
PLCIMPORTOPTIONS_REPLACE = 2
PLCIMPORTOPTIONS_SKIP = 3


# =========================
# 2) HELFER: Sicher schreiben
# =========================
def safe_write_text_file(path: str, content: str, encoding="utf-8"):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    tmp_path = path + ".tmp"

    with open(tmp_path, "w", encoding=encoding, newline="\n") as f:
        f.write(content)
        f.flush()
        os.fsync(f.fileno())

    os.replace(tmp_path, path)


# =========================
# 3) PARSE: TcPlcObject -> Daten rausziehen
# =========================
def parse_tcplcobject_xml(tc_xml_path: str):
    if not os.path.exists(tc_xml_path):
        raise FileNotFoundError(f"Quelle nicht gefunden: {tc_xml_path}")

    data = open(tc_xml_path, "rb").read()
    if data.startswith(b"\xef\xbb\xbf"):
        data = data[3:]

    root = ET.fromstring(data)

    ns = {
        "tc": "http://www.beckhoff.com/plcopen/xml/tc",
        "p": "http://www.plcopen.org/xml/tc6_0200",
    }

    pou = root.find("p:POU", ns)
    if pou is None:
        raise ValueError("Kein <POU> gefunden in TcPlcObject Datei.")

    pou_name = pou.attrib.get("Name", "FB_GeneratedJob_01")

    decl_node = pou.find("p:Declaration", ns)
    impl_node = pou.find("p:Implementation/p:ST", ns)

    pou_decl_text = (decl_node.text or "").strip()
    pou_body_text = (impl_node.text or "").strip()

    methods = []
    for m in pou.findall("p:Method", ns):
        m_name = m.attrib.get("Name", "Method")
        m_decl = (m.find("p:Declaration", ns).text or "").strip()
        m_code = (m.find("p:Implementation/p:ST", ns).text or "").strip()
        methods.append((m_name, m_decl, m_code))

    return pou_name, pou_decl_text, pou_body_text, methods


# =========================
# 4) TRANSFORM: State Enum -> INT State
# =========================
def normalize_state_machine_code(code: str) -> str:
    if not code:
        return code

    out = code
    out = out.replace("eState := RUNNING;", "eState := 1;")
    out = out.replace("eState := IDLE;", "eState := 0;")
    out = out.replace("eState := DONE;", "eState := 2;")
    out = out.replace("eState := ERROR;", "eState := 3;")
    out = out.replace("eState := ABORTED;", "eState := 4;")

    out = out.replace("\n    IDLE:", "\n    0:")
    out = out.replace("\n    RUNNING:", "\n    1:")
    out = out.replace("\n    DONE:", "\n    2:")
    out = out.replace("\n    ERROR:", "\n    3:")
    out = out.replace("\n    ABORTED:", "\n    4:")

    out = out.replace("\nIDLE:", "\n0:")
    out = out.replace("\nRUNNING:", "\n1:")
    out = out.replace("\nDONE:", "\n2:")
    out = out.replace("\nERROR:", "\n3:")
    out = out.replace("\nABORTED:", "\n4:")

    return out


# =========================
# 5) BUILD: PLCopen <project> XML korrekt erzeugen
# =========================
def build_plcopen_project_xml(pou_name: str, methods, pou_body_comment: str) -> str:
    PLC_NS = "http://www.plcopen.org/xml/tc6_0200"
    XHTML_NS = "http://www.w3.org/1999/xhtml"

    ET.register_namespace("", PLC_NS)
    ET.register_namespace("xhtml", XHTML_NS)

    project = ET.Element(f"{{{PLC_NS}}}project")

    # fileHeader (muss drin sein)
    fileHeader = ET.SubElement(project, f"{{{PLC_NS}}}fileHeader")
    fileHeader.set("companyName", "GeneratedByPython")
    fileHeader.set("productName", "TwinCAT PLC Control")
    fileHeader.set("productVersion", "3.5.x")
    fileHeader.set("creationDateTime", time.strftime("%Y-%m-%dT%H:%M:%S"))

    # contentHeader (TwinCAT braucht coordinateInfo!)
    contentHeader = ET.SubElement(project, f"{{{PLC_NS}}}contentHeader")
    contentHeader.set("name", "Proj1")
    contentHeader.set("modificationDateTime", time.strftime("%Y-%m-%dT%H:%M:%S"))

    # ✅ coordinateInfo genauso wie TwinCAT Export
    coordinateInfo = ET.SubElement(contentHeader, f"{{{PLC_NS}}}coordinateInfo")

    fbd = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}fbd")
    ET.SubElement(fbd, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})

    ld = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}ld")
    ET.SubElement(ld, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})

    sfc = ET.SubElement(coordinateInfo, f"{{{PLC_NS}}}sfc")
    ET.SubElement(sfc, f"{{{PLC_NS}}}scaling", {"x": "1", "y": "1"})

    # optional: addData in contentHeader (TwinCAT export hat das drin)
    addData_ch = ET.SubElement(contentHeader, f"{{{PLC_NS}}}addData")
    data_pi = ET.SubElement(addData_ch, f"{{{PLC_NS}}}data")
    data_pi.set("name", "http://www.3s-software.com/plcopenxml/projectinformation")
    data_pi.set("handleUnknown", "implementation")
    ET.SubElement(data_pi, f"{{{PLC_NS}}}ProjectInformation")

    # types
    types = ET.SubElement(project, f"{{{PLC_NS}}}types")
    ET.SubElement(types, f"{{{PLC_NS}}}dataTypes")
    pous = ET.SubElement(types, f"{{{PLC_NS}}}pous")

    # POU
    pou = ET.SubElement(pous, f"{{{PLC_NS}}}pou")
    pou.set("name", pou_name)
    pou.set("pouType", "functionBlock")

    # Interface
    interface = ET.SubElement(pou, f"{{{PLC_NS}}}interface")
    localVars = ET.SubElement(interface, f"{{{PLC_NS}}}localVars")

    def add_var(name: str, typename: str):
        var = ET.SubElement(localVars, f"{{{PLC_NS}}}variable")
        var.set("name", name)
        t = ET.SubElement(var, f"{{{PLC_NS}}}type")
        ET.SubElement(t, f"{{{PLC_NS}}}{typename}")

    # Robust: INT statt Inline-Enum
    add_var("eState", "INT")
    add_var("bFinished", "BOOL")
    add_var("nCounter", "INT")
    add_var("fResult", "LREAL")

    # Body (optional)
    body = ET.SubElement(pou, f"{{{PLC_NS}}}body")
    st = ET.SubElement(body, f"{{{PLC_NS}}}ST")
    xhtml = ET.SubElement(st, f"{{{XHTML_NS}}}xhtml")
    xhtml.text = pou_body_comment or "// FB Body"

    # Methoden als AddData (3S-Erweiterung)
    addData = ET.SubElement(pou, f"{{{PLC_NS}}}addData")

    for m_name, m_decl, m_code in methods:
        ret_type = "BOOL"
        if ":" in m_decl:
            ret_type = m_decl.split(":")[-1].strip().split()[0].strip().upper()

        m_code_norm = normalize_state_machine_code(m_code)

        # Rückgabewert absichern
        if ret_type == "BOOL" and f"{m_name} :=" not in m_code_norm:
            m_code_norm += f"\n{m_name} := TRUE;"
        if ret_type == "INT" and f"{m_name} :=" not in m_code_norm:
            m_code_norm += f"\n{m_name} := eState;"

        data = ET.SubElement(addData, f"{{{PLC_NS}}}data")
        data.set("name", "http://www.3s-software.com/plcopenxml/method")
        data.set("handleUnknown", "implementation")

        meth = ET.SubElement(data, f"{{{PLC_NS}}}Method")
        meth.set("name", m_name)
        meth.set("ObjectId", str(uuid.uuid4()))

        m_interface = ET.SubElement(meth, f"{{{PLC_NS}}}interface")
        returnType = ET.SubElement(m_interface, f"{{{PLC_NS}}}returnType")
        ET.SubElement(returnType, f"{{{PLC_NS}}}{ret_type}")

        m_body = ET.SubElement(meth, f"{{{PLC_NS}}}body")
        m_st = ET.SubElement(m_body, f"{{{PLC_NS}}}ST")
        m_xhtml = ET.SubElement(m_st, f"{{{XHTML_NS}}}xhtml")
        m_xhtml.text = m_code_norm

    # instances (TwinCAT export hat das drin; als leeres Minimum hilft es oft)
    instances = ET.SubElement(project, f"{{{PLC_NS}}}instances")
    ET.SubElement(instances, f"{{{PLC_NS}}}configurations")

    xml_bytes = ET.tostring(project, encoding="utf-8", xml_declaration=True)
    return xml_bytes.decode("utf-8")


# =========================
# 6) IMPORT: TwinCAT PlcOpenImport
# =========================
def import_plcopen_into_twincat(sln_path: str, plcopen_xml_path: str):
    if not os.path.exists(sln_path):
        print(f"❌ Solution nicht gefunden: {sln_path}")
        return
    if not os.path.exists(plcopen_xml_path):
        print(f"❌ PLCopen Datei nicht gefunden: {plcopen_xml_path}")
        return

    print(f"Starte Automation Interface ({DTE_PROGID})...")

    try:
        try:
            dte = win32com.client.GetActiveObject(DTE_PROGID)
        except Exception:
            dte = win32com.client.Dispatch(DTE_PROGID)
            dte.MainWindow.Visible = True
        dte.UserControl = True
    except Exception as e:
        print(f"❌ DTE Start fehlgeschlagen: {e}")
        return

    # Solution öffnen
    try:
        current_sln = dte.Solution.FullName
        if current_sln.lower() != sln_path.lower():
            print("Öffne Solution...")
            dte.Solution.Open(sln_path)
            while not dte.Solution.IsOpen:
                time.sleep(1)
    except Exception as e:
        print(f"❌ Fehler beim Öffnen der Solution: {e}")
        return

    # TwinCAT Projekt finden
    print("Suche TwinCAT Projekt...")
    tc_project = None
    for i in range(1, dte.Solution.Projects.Count + 1):
        try:
            p = dte.Solution.Projects.Item(i)
            if p.FullName.lower().endswith(".tsproj"):
                tc_project = p
                print(f"Gefunden: {p.Name}")
                break
        except:
            continue

    if not tc_project:
        print("❌ Kein .tsproj gefunden.")
        return

    # SystemManager
    try:
        sys_mgr = tc_project.Object
    except Exception as e:
        print(f"❌ Fehler SystemManager Zugriff: {e}")
        return

    # PLC Root
    print("Suche PLC Root (TIPC)...")
    try:
        root_plc = sys_mgr.LookupTreeItem("TIPC")
    except Exception as e:
        print(f"❌ PLC Root 'TIPC' nicht gefunden: {e}")
        return

    # SPS Projekt
    print("Suche SPS-Projekt...")
    plc_project_item = None
    for i in range(1, root_plc.ChildCount + 1):
        child = root_plc.Child(i)
        try:
            if child.NestedProject:
                plc_project_item = child.NestedProject
                print(f"SPS-Projekt gefunden: {child.Name}")
                break
        except:
            pass

    if not plc_project_item:
        print("❌ Kein SPS-Projekt gefunden.")
        return

    # Import
    print("Starte PLCopen Import (PlcOpenImport)...")
    try:
        plc_project_item.PlcOpenImport(plcopen_xml_path, PLCIMPORTOPTIONS_REPLACE, "", True)
        print("✅ ✅ ✅ Import erfolgreich ✅ ✅ ✅")
    except Exception as e:
        print(f"❌ PlcOpenImport fehlgeschlagen: {e}")
        print("Hinweis: Manuell in TwinCAT testen: Rechtsklick PLC-Projekt oder POUs → Import PLCopenXML…")


# =========================
# 7) MAIN
# =========================
print("Lese TcPlcObject XML...")
pou_name, pou_decl, pou_body, methods = parse_tcplcobject_xml(SOURCE_TC_OBJECT_XML)

print("Erzeuge PLCopen <project> XML...")
plcopen_xml = build_plcopen_project_xml(pou_name, methods, pou_body)

print("Schreibe Datei nach:", OUT_XML_PATH)
safe_write_text_file(OUT_XML_PATH, plcopen_xml)

print("Check Head:")
with open(OUT_XML_PATH, "r", encoding="utf-8") as f:
    print(f.read(350))

import_plcopen_into_twincat(SLN_PATH, OUT_XML_PATH)


Lese TcPlcObject XML...
Erzeuge PLCopen <project> XML...
Schreibe Datei nach: D:\MA_Python_Agent\Notebooks\Generierte_PLCOpenXML\FB_GeneratedJob_01.xml
Check Head:
<?xml version='1.0' encoding='utf-8'?>
<project xmlns="http://www.plcopen.org/xml/tc6_0200" xmlns:xhtml="http://www.w3.org/1999/xhtml"><fileHeader companyName="GeneratedByPython" productName="TwinCAT PLC Control" productVersion="3.5.x" creationDateTime="2026-01-19T08:55:05" /><contentHeader name="Proj1" modificationDateTime="2026-01-19T08:55:05"><c
Starte Automation Interface (TcXaeShell.DTE.17.0)...
Suche TwinCAT Projekt...
Gefunden: TestProjektTwinCATEvents
Suche PLC Root (TIPC)...
Suche SPS-Projekt...
SPS-Projekt gefunden: Proj1
Starte PLCopen Import (PlcOpenImport)...
✅ ✅ ✅ Import erfolgreich ✅ ✅ ✅


In [ ]:
import os
import time
import xml.etree.ElementTree as ET
import win32com.client

# =========================
# KONFIGURATION
# =========================
DTE_PROGID = "TcXaeShell.DTE.17.0"

SLN_PATH = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln"
SOURCE_TC_OBJECT_XML = r"C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\FB_GeneratedJob_01.xml"

PLC_PROJECT_NAME = "Proj1"
POU_NAME = "FB_GeneratedJob_01"

# ✅ RICHTIG: ST = 1 (nicht 6)
IECLANGUAGE_ST_NUM = 1
IECLANGUAGE_ST_STR = "ST"  # Beckhoff nutzt oft "ST" als String in vInfo :contentReference[oaicite:3]{index=3}


# =========================
# HELFER: DTE / Solution
# =========================
def _get_or_start_dte(progid: str):
    try:
        dte = win32com.client.GetActiveObject(progid)
    except Exception:
        dte = win32com.client.Dispatch(progid)
        dte.MainWindow.Visible = True
    dte.UserControl = True
    return dte


def _open_solution(dte, sln_path: str):
    if not os.path.exists(sln_path):
        raise FileNotFoundError(f"Solution nicht gefunden: {sln_path}")

    if (dte.Solution is None) or (not dte.Solution.IsOpen) or (dte.Solution.FullName.lower() != sln_path.lower()):
        print("Öffne Solution...")
        dte.Solution.Open(sln_path)

        for _ in range(60):
            if dte.Solution.IsOpen:
                break
            time.sleep(0.5)

    print("✅ Solution offen:", dte.Solution.FullName)


def _find_tsproj(dte):
    print("Suche TwinCAT Projekt (.tsproj)...")
    for i in range(1, dte.Solution.Projects.Count + 1):
        try:
            p = dte.Solution.Projects.Item(i)
            if p.FullName and p.FullName.lower().endswith(".tsproj"):
                print("✅ Gefunden:", p.Name)
                return p
        except Exception:
            pass
    return None


def _lookup_tree_item(sys_mgr, candidates):
    for c in candidates:
        try:
            return sys_mgr.LookupTreeItem(c)
        except Exception:
            continue
    return None


# =========================
# PARSER: TcPlcObject XML lesen
# =========================
def parse_tcplcobject_xml(tc_xml_path: str):
    if not os.path.exists(tc_xml_path):
        raise FileNotFoundError(f"Quelle nicht gefunden: {tc_xml_path}")

    data = open(tc_xml_path, "rb").read()
    if data.startswith(b"\xef\xbb\xbf"):
        data = data[3:]

    root = ET.fromstring(data)

    ns = {"p": "http://www.plcopen.org/xml/tc6_0200"}

    pou = root.find("p:POU", ns)
    if pou is None:
        raise ValueError("Kein <POU> gefunden in TcPlcObject Datei.")

    pou_name = pou.attrib.get("Name", POU_NAME)

    decl_node = pou.find("p:Declaration", ns)
    pou_decl = (decl_node.text or "").strip()

    impl_node = pou.find("p:Implementation/p:ST", ns)
    pou_impl = (impl_node.text or "").strip()

    methods = []
    for m in pou.findall("p:Method", ns):
        m_name = m.attrib.get("Name", "Method")
        m_decl = (m.find("p:Declaration", ns).text or "").strip()
        m_code = (m.find("p:Implementation/p:ST", ns).text or "").strip()
        methods.append((m_name, m_decl, m_code))

    return pou_name, pou_decl, pou_impl, methods


def extract_method_return_type(method_decl: str) -> str:
    # Beispiel: "METHOD JobStart : BOOL"
    if ":" in method_decl:
        return method_decl.split(":")[-1].strip().split()[0].strip().upper()
    return "BOOL"


# =========================
# ERZEUGEN: FB + Methoden in POUs
# =========================
def create_fb_with_methods_in_pous():
    print("Lese TcPlcObject XML...")
    pou_name, pou_decl, pou_impl, methods = parse_tcplcobject_xml(SOURCE_TC_OBJECT_XML)

    print("Starte Automation Interface...")
    dte = _get_or_start_dte(DTE_PROGID)
    _open_solution(dte, SLN_PATH)

    tc_project = _find_tsproj(dte)
    if not tc_project:
        raise RuntimeError("❌ Kein .tsproj gefunden.")

    sys_mgr = tc_project.Object

    # POUs Ordner holen
    pous_item = _lookup_tree_item(sys_mgr, [
        f"TIPC^{PLC_PROJECT_NAME}^{PLC_PROJECT_NAME} Projekt^POUs",
        f"TIPC^{PLC_PROJECT_NAME}^{PLC_PROJECT_NAME} Project^POUs",
    ])
    if not pous_item:
        raise RuntimeError("❌ Ordner 'POUs' wurde nicht gefunden.")

    print("✅ Ordner POUs gefunden.")

    # Falls FB existiert: löschen
    existing_fb = None
    try:
        for i in range(1, pous_item.ChildCount + 1):
            c = pous_item.Child(i)
            if getattr(c, "Name", "") == pou_name:
                existing_fb = c
                break
    except Exception:
        pass

    if existing_fb:
        print(f"Altes Objekt '{pou_name}' gefunden -> Lösche es...")
        existing_fb.Delete()
        time.sleep(0.2)

    # ✅ Function Block erstellen (Subtype 604) :contentReference[oaicite:4]{index=4}
    # vInfo kann "ST" sein oder leer -> Default ST
    print(f"Erstelle Function Block '{pou_name}' direkt in POUs...")
    new_fb = pous_item.CreateChild(pou_name, 604, "", IECLANGUAGE_ST_STR)

    # Declaration / Implementation setzen
    try:
        new_fb.DeclarationText = pou_decl
    except Exception as e:
        print("⚠️ Konnte DeclarationText nicht setzen:", e)

    try:
        if pou_impl:
            new_fb.ImplementationText = pou_impl
    except Exception as e:
        print("⚠️ Konnte ImplementationText nicht setzen:", e)

    # ✅ Methoden unter FB erstellen: SubType 609 (Method POU) :contentReference[oaicite:5]{index=5}
    for (m_name, m_decl, m_code) in methods:
        ret_type = extract_method_return_type(m_decl)
        print(f"Erstelle Methode '{m_name}' (Return: {ret_type}) ...")

        # vInfo laut Beckhoff:
        # [0]=Language optional ("ST")
        # [1]=Return type (z.B. BOOL)
        # [2]=Access modifier optional (z.B. PUBLIC)
        vinfo = [IECLANGUAGE_ST_STR, ret_type, "PUBLIC"]

        method_item = new_fb.CreateChild(m_name, 609, "", vinfo)

        # Methoden Declaration + Implementation setzen
        try:
            method_item.DeclarationText = m_decl
        except Exception as e:
            print(f"⚠️ Konnte DeclarationText für Methode {m_name} nicht setzen:", e)

        try:
            method_item.ImplementationText = m_code
        except Exception as e:
            print(f"⚠️ Konnte ImplementationText für Methode {m_name} nicht setzen:", e)

    # SaveAll
    try:
        dte.ExecuteCommand("File.SaveAll")
        print("✅ SaveAll ausgeführt.")
    except Exception:
        print("⚠️ Konnte SaveAll nicht ausführen (nicht kritisch).")

    print(f"✅ Fertig: '{pou_name}' ist jetzt unter POUs mit Methoden erstellt.")


# AUSFÜHREN
create_fb_with_methods_in_pous()


Lese TcPlcObject XML...
Starte Automation Interface...
✅ Solution offen: C:\Users\Alexander Verkhov\OneDrive\Dokumente\MPA\TestProjektTwinCATEvents\TestProjektTwinCATEvents.sln
Suche TwinCAT Projekt (.tsproj)...
✅ Gefunden: TestProjektTwinCATEvents
✅ Ordner POUs gefunden.
Erstelle Function Block 'FB_GeneratedJob_01' direkt in POUs...
⚠️ Konnte ImplementationText nicht setzen: (-2147352567, 'Ausnahmefehler aufgetreten.', (0, 'System.Xml', 'Ungültige Daten auf Stammebene. Zeile 1, Position 1.', None, 0, -2146232000), None)
Erstelle Methode 'JobStart' (Return: BOOL) ...


com_error: (-2147352567, 'Ausnahmefehler aufgetreten.', (0, 'TwinCAT.XAE.Automation.17.0', "TwinCAT PLC automation call (ITcSmTreeItem:CreateChild) failed: Kann 'JobStart' nicht unterhalb von 'FB_GeneratedJob_01 [TestProjektTwinCATEvents: PLC: Proj1]' einfügen.", None, 0, -2147467259), None)